In [1]:
import pandas as pd
import numpy as np
import random
import math
import optuna

In [2]:
def BALANCE(weights):
    #Making sure the total sum of the weights eual to 1
    weights = [w/sum(weights) for w in weights] # Making sure all weights represent proportions that add up to 1
    return weights

In [3]:
def ratio(a,b,c):                         #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
    #calculating sharpe ratio
    return (a-c)/b

In [4]:
def number_of_years(y):        #calculates the number of years of the dataset
  p=y.index[0]                 #date of first row in the dataset (datetime format)
  q=y.index[len(y)-1]          #date of last row in the dataset  (datetime format)
  return ((q-p).days+1)/365

In [5]:
df=pd.read_csv('n50.csv',parse_dates=['Date'],index_col='Date')  #Importing Dataset
df = df.loc["2016-01-01" : ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)

In [6]:

def antcolonyHPTuning(ITERATIONS,Q,EVA_RATE,ANTS):

    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')  # Importing Dataset
    df = df.loc['2016-01-01':]  # Since 2016-01-01, 5y(1234rows till 2020-12-31)
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)

    trading_days = len(df) / number_of_years(tdf)

    returnsh = df.pct_change()  # Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0, inplace=True)  # calculating daily returns of the stocks in the portfolio

    returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
    for cols in returnso.columns.tolist():
        for i in range(0, len(df)):
            if returnso[cols][i] > 0:
                returnso[cols][i] = 0

            # Here, returnso would mean return considered for sortino ratio

    covmatsh = returnsh.cov() * trading_days
    covmatso = returnso.cov() * trading_days

    risk_free_rate = 0.0358
    stocks = df.shape[1]

    global_warr_sortino = []
    global_war_sharpe = []

    pbest = -1

# Initializing the current fitness value

    fitness = 0

# for each iteration

    for iteration in range(ITERATIONS):

    # PREPARAING THE PHEROMONE MATRIX WHERE THE COLS=STOCKS AND  ROWS=ANTS

        pheromon = [[0] * stocks for i in range(ANTS + 1)]  # why (ants+1)?The last ant can update the pheromone values in the last row

    # Initializing the pheromone status

        for i in range(len(pheromon[0])):
            pheromon[0][i] = random.randint(1, 15)  # When input stocks varies, this needs to vary accordingly.(Divide number of stocks / 2)

    # copying the values and storing it in temp_pher

        temp_pher = pheromon[0]

    # Making sure that the total amount of pheromone equals 1

        weights = np.array(BALANCE(temp_pher))

    # calculating annulaised portfolio return

        returns_temp = np.sum(returnsh.mean() * weights) * trading_days

    # print(returns_temp)

    # calculating portfolio varience wrt calculating sharpe ratio

        varsh = np.dot(weights.T, np.dot(covmatsh, weights))

    # print(varsh)

    # portfolio risk

        volatility_temp = np.sqrt(varsh)

    # Calculating fitness value(ie sharpe ratio)

        fitness = ratio(returns_temp, volatility_temp, risk_free_rate)

    # Initializing the intial fitness value as the best fitness value(pbest)

        if pbest == -1:
            pbest = fitness

    # list

        path = []

    # for each ant

        for ant in range(ANTS - 1):

        # find the total pheromone
        # print("Pheronome: ",pheromon[ant])

            total = sum(pheromon[ant])

        # print("Total: ",total)

        # Initializing probability

            probability = (pheromon[ant])[:]

        # print("Probability: ",probability)

        # finding probability of each stocks pheromone

            for p in range(len(probability)):
                probability[p] = probability[p] / total

        # Trying to select stocks in decreasing order based on their pheromone level and storing the stock order in a list(path)

            for stock in range(stocks):
                select = probability.index(max(probability))
                probability[select] = -math.inf
                path.append(select)

            # print("Path: ",path)

        # Updating the pheromone level of each stock for the next ant
        # Formula: old pheromone level * (1-eva_rate) + Q * (fitness/pbest) where Q is fixed amount of pheromone

            for s in path:
                pheromon[ant + 1][s] = pheromon[ant][s] * (1
                        - EVA_RATE) + Q * (fitness / pbest)

        # making sure that the updated pheromon adds upto 1

            temp_pher = pheromon[ant + 1]
            weights = np.array(BALANCE(temp_pher))
            returns_temp = np.sum(returnsh.mean() * weights) \
                * trading_days  # calculating annulaised portfolio return
            varsh = np.dot(weights.T, np.dot(covmatsh, weights))  # calculating portfolio varience wrt calculating sharpe ratio
            volatility_temp = np.sqrt(varsh)  # portfolio risk
            fitness = ratio(returns_temp, volatility_temp,
                            risk_free_rate)  # calculating sharpe ratio

        # print(fitness)
        # comparing the old fitness value with the updated fitness value

            if fitness > pbest:

            # if the updated fitness value is better than the previous, change pbest to present fitness value

                pbest = fitness

            # print(pbest)
            # remembering the weights of the best portfolio

                global_warr_sharpe = weights.tolist()

        # print("Global warr sharpe: ",global_warr_sharpe)

    return pbest


In [7]:
def objective(trial):
    ITERATIONS=trial.suggest_uniform('ITERATIONS',10,50)
    EVA_RATE=trial.suggest_uniform('EVA_RATE',0.1,1)
    Q=trial.suggest_uniform('Q',0.1,1)
    ANTS=trial.suggest_uniform('ANTS',10,20)
    return antcolonyHPTuning(int(ITERATIONS),Q,EVA_RATE,int(ANTS))

In [8]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=10)


[I 2021-11-19 21:47:21,914] A new study created in memory with name: no-name-1a06788c-f1ef-4a2c-82e1-ee1c300e53f5
[I 2021-11-19 21:47:23,236] Trial 0 finished with value: 1.3536411050741 and parameters: {'ITERATIONS': 28.182098283449687, 'EVA_RATE': 0.3693514084350943, 'Q': 0.33121728084146795, 'ANTS': 14.352237083912406}. Best is trial 0 with value: 1.3536411050741.
[I 2021-11-19 21:47:24,408] Trial 1 finished with value: 1.351116985840663 and parameters: {'ITERATIONS': 22.150733623162246, 'EVA_RATE': 0.9325234273797488, 'Q': 0.8120433617796824, 'ANTS': 10.987303964101539}. Best is trial 0 with value: 1.3536411050741.
[I 2021-11-19 21:47:25,742] Trial 2 finished with value: 1.3711038032056169 and parameters: {'ITERATIONS': 32.782942377080275, 'EVA_RATE': 0.6221089103313002, 'Q': 0.424557999090716, 'ANTS': 14.125364868665942}. Best is trial 2 with value: 1.3711038032056169.
[I 2021-11-19 21:47:26,896] Trial 3 finished with value: 1.3182339074816203 and parameters: {'ITERATIONS': 16.707

In [9]:
best=study.best_params

In [10]:
best

{'ITERATIONS': 36.70936386604793,
 'EVA_RATE': 0.1016457195654718,
 'Q': 0.1619195710339478,
 'ANTS': 10.616811587154958}

In [11]:
ITERATIONS=int(best['ITERATIONS'])
Q=best['Q']
EVA_RATE=best['EVA_RATE']
ANTS=int(best['ANTS'])


In [12]:
def ACO_Sharpe(ITERATIONS,Q,EVA_RATE,ANTS):

    top_portfolios = []
    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')  # Importing Dataset
    df = df.loc['2016-01-01':]  # Since 2016-01-01, 5y(1234rows till 2020-12-31)
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)

    trading_days = len(df) / number_of_years(tdf)

    returnsh = df.pct_change()  # Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0, inplace=True)  # calculating daily returns of the stocks in the portfolio

    returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
    for cols in returnso.columns.tolist():
        for i in range(0, len(df)):
            if returnso[cols][i] > 0:
                returnso[cols][i] = 0

            # Here, returnso would mean return considered for sortino ratio

    covmatsh = returnsh.cov() * trading_days
    covmatso = returnso.cov() * trading_days

    risk_free_rate = 0.0358
    stocks = df.shape[1]

    global_warr_sortino = []
    global_warr_sharpe = []

    pbest = -1

# Initializing the current fitness value

    fitness = 0

# for each iteration

    for iteration in range(ITERATIONS):

    # PREPARAING THE PHEROMONE MATRIX WHERE THE COLS=STOCKS AND  ROWS=ANTS

        pheromon = [[0] * stocks for i in range(ANTS + 1)]  # why (ants+1)?The last ant can update the pheromone values in the last row

    # Initializing the pheromone status

        for i in range(len(pheromon[0])):
            pheromon[0][i] = random.randint(1, 15)  # When input stocks varies, this needs to vary accordingly.(Divide number of stocks / 2)

    # copying the values and storing it in temp_pher

        temp_pher = pheromon[0]

    # Making sure that the total amount of pheromone equals 1

        weights = np.array(BALANCE(temp_pher))

    # calculating annulaised portfolio return

        returns_temp = np.sum(returnsh.mean() * weights) * trading_days

    # print(returns_temp)

    # calculating portfolio varience wrt calculating sharpe ratio

        varsh = np.dot(weights.T, np.dot(covmatsh, weights))

    # print(varsh)

    # portfolio risk

        volatility_temp = np.sqrt(varsh)

    # Calculating fitness value(ie sharpe ratio)

        fitness = ratio(returns_temp, volatility_temp, risk_free_rate)

    # Initializing the intial fitness value as the best fitness value(pbest)

        if pbest == -1:
            pbest = fitness

    # list

        path = []

    # for each ant

        for ant in range(ANTS - 1):

        # find the total pheromone
        # print("Pheronome: ",pheromon[ant])

            total = sum(pheromon[ant])

        # print("Total: ",total)

        # Initializing probability

            probability = (pheromon[ant])[:]

        # print("Probability: ",probability)

        # finding probability of each stocks pheromone

            for p in range(len(probability)):
                probability[p] = probability[p] / total

        # Trying to select stocks in decreasing order based on their pheromone level and storing the stock order in a list(path)

            for stock in range(stocks):
                select = probability.index(max(probability))
                probability[select] = -math.inf
                path.append(select)

            # print("Path: ",path)

        # Updating the pheromone level of each stock for the next ant
        # Formula: old pheromone level * (1-eva_rate) + Q * (fitness/pbest) where Q is fixed amount of pheromone

            for s in path:
                pheromon[ant + 1][s] = pheromon[ant][s] * (1
                        - EVA_RATE) + Q * (fitness / pbest)

        # making sure that the updated pheromon adds upto 1

            temp_pher = pheromon[ant + 1]
            weights = np.array(BALANCE(temp_pher))
            returns_temp = np.sum(returnsh.mean() * weights) \
                * trading_days  # calculating annulaised portfolio return
            varsh = np.dot(weights.T, np.dot(covmatsh, weights))  # calculating portfolio varience wrt calculating sharpe ratio
            volatility_temp = np.sqrt(varsh)  # portfolio risk
            fitness = ratio(returns_temp, volatility_temp,
                            risk_free_rate)  # calculating sharpe ratio

        # print(fitness)
        # comparing the old fitness value with the updated fitness value

            if fitness > pbest:

            # if the updated fitness value is better than the previous, change pbest to present fitness value

                pbest = fitness

                # portfolio content

                portfolio_returns = returns_temp
                portfolio_risk = volatility_temp
                portfolio_sharpe = fitness
                portfolio_array = [portfolio_returns,portfolio_risk,portfolio_sharpe,weights.tolist()]
                top_portfolios.append(portfolio_array)

            # print(pbest)
            # remembering the weights of the best portfolio
                # global_warr_sharpe=weights.tolist()
        # print("Global warr sharpe: ",global_warr_sharpe)

    return top_portfolios


In [13]:
top_portfolios_sharpe=ACO_Sharpe(ITERATIONS,Q,EVA_RATE,ANTS)
#top_portfolios=antcolony_optimization(200,0.1,0.63,300)

In [14]:
#top_portfolios_sharpe_df=pd.DataFrame(top_portfolios_sharpe[0:2],columns=['Returns','Risk','Sharpe ratio'])
top_portfolios_sharpe_df=pd.DataFrame(top_portfolios_sharpe,columns=['Returns','Risk','Sharpe','Weights'])
for counter,symbol in enumerate(df.columns):
  top_portfolios_sharpe_df[symbol + " Weight"] = [Weight[counter] for Weight in top_portfolios_sharpe_df]

ValueError: Length of values (4) does not match length of index (12)

In [ ]:
top_portfolios_sharpe_df

In [ ]:
#sorting portfolio based on sharpe ratio
top_portfolios_sharpe.sort(key=lambda x:x[3],reverse=True)


In [ ]:
#index of portfolio parameters 
weights_index=0
return_index=1
risk_index=2
sharpe_index=3


iter_portfolio=10 if len(top_portfolios_sharpe)>10 else len(top_portfolios_sharpe)
        
for i in range(0,iter_portfolio):
  if(i==0):
    names=df.columns
    j=0
    print("******** OPTIMAL PORTFOLIO *********\n")
    print("************ WEIGHTS *************")
    for x in (names):
      print("{} -> {} %".format(x,top_portfolios_sharpe[i][weights_index][j]*100))
      j+=1
    print("\n")
    print("***** OPTIMAL PORTFOLIO COMPONENTS *****")
    print("Returns -> {} %".format(top_portfolios_sharpe[i][return_index]*100))
    print("Risk -> {} %".format(top_portfolios_sharpe[i][risk_index]*100))
    print("Sharpe Ratio -> {}".format(top_portfolios_sharpe[i][sharpe_index]))
    print("\n")
  else:
    print("\n******* PORTFOLIO {}. ******".format(i+1))
    names=df.columns
    print("Returns -> {} %".format(top_portfolios_sharpe[i][return_index]*100))
    print("Risk -> {} %".format(top_portfolios_sharpe[i][risk_index]*100))
    print("Sharpe Ratio -> {}".format(top_portfolios_sharpe[i][sharpe_index]))




In [ ]:
def ACO_Sortino(ITERATIONS,Q,EVA_RATE,ANTS):
    pbest=-1
    fitness=0
    top_portfolios_sortino=[]
    df=pd.read_csv('n50.csv',parse_dates=['Date'],index_col='Date')  #Importing Dataset
    df = df.loc["2016-01-01" : ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
    tdf=df.copy()                  #deep copy
    df.reset_index(drop=True, inplace=True)

    trading_days=len(df)/number_of_years(tdf)


    returnsh=df.pct_change()                  #Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0,inplace=True)           #calculating daily returns of the stocks in the portfolio

    returnso=returnsh.copy()                  #this cell considers only NEGATIVE returns so as to calculate sortino ratio
    for cols in returnso.columns.tolist():
        for i in range(0,len(df)):
            if returnso[cols][i] > 0:
                returnso[cols][i]=0
            #Here, returnso would mean return considered for sortino ratio

    covmatso=returnso.cov()*trading_days

    risk_free_rate = 0.0358
    stocks=df.shape[1]

    global_warr_sharpe=[]
    for iteration in range(ITERATIONS):
        pheromon=[[0]*stocks for i in range(ANTS+1)]
        for i in range(len(pheromon[0])):
            pheromon[0][i]=random.randint(1,15)
        temp_pher=pheromon[0]
        weights=np.array(BALANCE(temp_pher))
        returns_temp = np.sum(returnsh.mean()*weights)*trading_days
        varso=np.dot(weights.T,np.dot(covmatso,weights))
        semi_temp = np.sqrt(varso)
        fitness = ratio(returns_temp,semi_temp,risk_free_rate)
        if pbest==-1:
            pbest=fitness
        path=[]
        for ant in range(ANTS-1):
            total=sum(pheromon[ant])
            probability=pheromon[ant][:]
            for p in range(len(probability)):
                probability[p]=(probability[p]/total)
            for stock in range(stocks):
                select=probability.index(max(probability))
                probability[select]=-math.inf
                path.append(select)
            for s in path:
                pheromon[ant+1][s]=pheromon[ant][s]*(1-EVA_RATE)+Q*(fitness/pbest)
            temp_pher=pheromon[ant+1]
            weights=np.array(BALANCE(temp_pher))
            returns_temp = np.sum(returnsh.mean()*weights)*trading_days
            varso=np.dot(weights.T,np.dot(covmatso,weights))
            semi_temp = np.sqrt(varso)
            fitness = ratio(returns_temp,semi_temp,risk_free_rate)
            if(fitness>pbest):
                pbest=fitness
                portfolio_returns=returns_temp
                portfolio_risk=semi_temp
                portfolio_sortino=fitness
                portfolio_array=[weights.tolist(),portfolio_returns,portfolio_risk,portfolio_sortino]
                top_portfolios_sortino.append(portfolio_array)
    return top_portfolios_sortino


In [ ]:
top_portfolios_sortino=ACO_Sortino(ITERATIONS,Q,EVA_RATE,ANTS)

In [ ]:
top_portfolios_sortino.sort(key=lambda x:x[3],reverse=True)

In [ ]:
#index of portfolio parameters 
weights_index=0
return_index=1
risk_index=2
sharpe_index=3


iter_portfolio=10 if len(top_portfolios_sortino)>10 else len(top_portfolios_sortino)
        
for i in range(0,iter_portfolio):
  if(i==0):
    names=df.columns
    j=0
    print("******** OPTIMAL PORTFOLIO *********\n")
    print("************ WEIGHTS *************")
    for x in (names):
      print("{} -> {} %".format(x,top_portfolios_sortino[i][weights_index][j]*100))
      j+=1
    print("\n")
    print("***** OPTIMAL PORTFOLIO COMPONENTS *****")
    print("Returns -> {} %".format(top_portfolios_sortino[i][return_index]*100))
    print("Risk -> {} %".format(top_portfolios_sortino[i][risk_index]*100))
    print("Sortino Ratio -> {}".format(top_portfolios_sortino[i][sharpe_index]))
    print("\n")
  else:
    print("\n******* PORTFOLIO {}. ******".format(i+1))
    names=df.columns
    print("Returns -> {} %".format(top_portfolios_sortino[i][return_index]*100))
    print("Risk -> {} %".format(top_portfolios_sortino[i][risk_index]*100))
    print("Sortino Ratio -> {}".format(top_portfolios_sortino[i][sharpe_index]))